### Import Library

In [1]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from collections import Counter
from scipy.stats import  ks_2samp, chi2_contingency
import matplotlib.pyplot as plt
from scipy import stats
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Flatten, Dense, Dropout

import nest_asyncio
nest_asyncio.apply()

# تنظیم بذر تصادفی برای تکرارپذیری

np.random.seed(42)
tf.random.set_seed(42)
# تنظیم زمینه اجرایی محلی
tff.backends.native.set_local_execution_context()

C:\Users\TOP\anaconda3\envs\evn8\lib\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.18.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Create Model 

In [2]:
# تابع ساخت مدل MLP
def create_mnist_mlp_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))  # ورودی به شکل مسطح شده (28x28 = 784)
    model.add(Dropout(0.3))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))  # لایه خروجی با 10 کلاس (softmax)
    optimizer = Adam(learning_rate=0.01)
    # model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # کامپایل کردن مدل

    return model

### Load and Split

In [31]:
# لود کردن داده‌های MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# نرمال‌سازی داده‌ها
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# مسطح کردن تصاویر
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# کدگذاری لیبل‌ها
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# تقسیم داده‌ها بین 10 کلاینت
num_clients = 10
client_data = np.array_split(x_train, num_clients)
client_labels = np.array_split(y_train, num_clients)

### Bug Injection

In [32]:
# لیستی از کلاینت‌هایی که باید دوبار نرمال‌سازی شوند
clients_to_double_normalize = [0]  # کلاینت‌های 3,6 (با اندیس صفر شروع می‌شود)

# دوبار نرمال‌سازی کلاینت‌های مشخص‌شده
for client_index in clients_to_double_normalize:
    client_data[client_index] = client_data[client_index] / 255.0

# پوشه‌ای برای ذخیره پارامترهای کلاینت‌ها ایجاد می‌کنیم
if not os.path.exists('client_params'):
    os.makedirs('client_params')

### Models training & save

In [33]:
# آموزش مدل‌ها برای هر کلاینت و ذخیره پارامترها
client_models = [] #لیست برای ذخیره مدل‌های هر کلاینت

base_model = create_mnist_mlp_model()  # مدل پایه

#base_model.save_weights('base_model_weights.h5') # ذخیره وزن‌های مدل پایه

for i in range(num_clients):
    print(f"Training model for client {i+1}")    
    model=create_mnist_mlp_model()
    model.set_weights(base_model.get_weights())
    
    # بررسی یکسان بودن وزن‌ها قبل از آموزش
    # m1 = model.get_weights()
    # b1 = base_model.get_weights()
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # کامپایل کردن مدل
    # for index, (base_w, clone_w) in enumerate(zip(m1, b1)):
    #     if not np.array_equal(base_w, clone_w):
    #         print(f"Weights at index {index} are different.")
    #     else:
    #         print(f"Weights at index {index} are identical.")
    model.fit(client_data[i], client_labels[i], epochs=5, batch_size=32, verbose=1)
    #model.save_weights(f'client_params/client_{i+1}_weights.h5') # ذخیره وزن‌های هر کلاینت
    #print("**********",model.predict(x_test[:100].argmax(axis=1)))
    client_models.append(model)  # افزودن مدل به لیست
print("Training completed for all clients.")

Training model for client 1
Epoch 1/5
188/188 [==============================] - 1s 7ms/step - loss: 1.7547 - accuracy: 0.4307
Epoch 2/5
188/188 [==============================] - 1s 7ms/step - loss: 0.8341 - accuracy: 0.7257
Epoch 3/5
188/188 [==============================] - 1s 7ms/step - loss: 0.6179 - accuracy: 0.8108
Epoch 4/5
188/188 [==============================] - 1s 6ms/step - loss: 0.5142 - accuracy: 0.8487
Epoch 5/5
188/188 [==============================] - 1s 6ms/step - loss: 0.4509 - accuracy: 0.8643
Training model for client 2
Epoch 1/5
188/188 [==============================] - 1s 5ms/step - loss: 0.6452 - accuracy: 0.7978
Epoch 2/5
188/188 [==============================] - 1s 5ms/step - loss: 0.2675 - accuracy: 0.9180
Epoch 3/5
188/188 [==============================] - 1s 5ms/step - loss: 0.1824 - accuracy: 0.9413
Epoch 4/5
188/188 [==============================] - 1s 5ms/step - loss: 0.1370 - accuracy: 0.9568
Epoch 5/5
188/188 [==============================] - 

### Class

In [34]:
def delta_class(models, x_test):
    num_models = len(models)
    argmax_preds = [model.predict(x_test).argmax(axis=1) for model in models]   # پیش‌بینی‌ها برای هر مدل
    diffs_matrix = np.zeros((num_models, num_models), dtype=int)    # ماتریس مربعی برای ذخیره تفاوت‌ها
    for i in range(num_models):       # پر کردن ماتریس با تفاوت پیش‌بینی‌ها
        for j in range(i+1, num_models):
            diffs = np.sum(argmax_preds[i] != argmax_preds[j])  # تعداد تفاوت‌های پیش‌بینی بین مدل i و مدل j
            diffs_matrix[i, j] = diffs
            diffs_matrix[j, i] = diffs  # ماتریس متقارن است
    return diffs_matrix

# اندازه‌گیری زمان
start_time = time.time()
diff_class = delta_class(client_models, x_test)
print("Difference Matrix for Delta Class:")
print(diff_class)

# end_time = time.time()
# execution_time = end_time - start_time
# print(f"\nExecution time for delta_class: {execution_time:.4f} seconds")

Difference Matrix for Delta Class:
[[   0 1993 1877 1947 1974 1851 2071 1935 1918 1960]
 [1993    0  551  558  534  519  656  641  529  595]
 [1877  551    0  492  540  444  608  521  474  530]
 [1947  558  492    0  584  543  579  572  546  561]
 [1974  534  540  584    0  548  688  637  536  552]
 [1851  519  444  543  548    0  635  507  515  552]
 [2071  656  608  579  688  635    0  634  672  636]
 [1935  641  521  572  637  507  634    0  607  548]
 [1918  529  474  546  536  515  672  607    0  514]
 [1960  595  530  561  552  552  636  548  514    0]]


### Score

In [39]:
def delta_score(models, x_test, threshold):
    num_models = len(models)
    diffs_matrix = np.zeros((num_models, num_models))

    # پیش‌بینی خروجی‌های هر مدل
    predictions = [model.predict(x_test) for model in models]
    
    # محاسبه‌ی argmax برای هر مدل
    argmax_preds = [np.argmax(pred, axis=1) for pred in predictions]

    # مقایسه مدل‌ها دو به دو
    for i in range(num_models):
        for j in range(i + 1, num_models):
            diff_count = 0  # شمارنده اختلافات

            # مقایسه‌ی argmax ها
            argmax_diff = argmax_preds[i] != argmax_preds[j]
            diff_count += np.sum(argmax_diff)  # اضافه کردن اختلاف در argmax
            
            # مقایسه‌ی احتمالات برای پیش‌بینی‌های با argmax یکسان
            same_argmax_indices = np.where(argmax_preds[i] == argmax_preds[j])[0]
            
            for idx in same_argmax_indices:
                # مقایسه‌ی احتمالات در همان کلاس‌های argmax
                prob_i = round(predictions[i][idx][argmax_preds[i][idx]], 2)
                prob_j = round(predictions[j][idx][argmax_preds[j][idx]], 2)
                
                # محاسبه اختلاف بین پیش‌بینی‌ها
                prob_diff = abs(prob_i - prob_j)
                
                # بررسی اختلاف با آستانه threshold
                if prob_diff >= threshold:
                    diff_count += 1  # شمارش اختلاف در پیش‌بینی‌ها

            # ذخیره اختلافات در هر دو موقعیت (i, j) و (j, i) برای متقارن بودن
            diffs_matrix[i, j] = diff_count
            diffs_matrix[j, i] = diff_count

    return diffs_matrix

# اجرای تابع با تعیین آستانه threshold
diff_score = delta_score(client_models, x_test, threshold=0.1)
print("Distance Matrix for Delta Score:")
print(diff_score)

# start_time = time.time()
# end_time = time.time()
# execution_time = end_time - start_time
# print(f"\nExecution time for delta_score: {execution_time:.4f} seconds")


Distance Matrix for Delta Score:
[[   0. 2503. 2320. 2460. 2435. 2253. 2703. 2500. 2422. 2489.]
 [2503.    0. 1390. 1403. 1372. 1351. 1655. 1563. 1368. 1490.]
 [2320. 1390.    0. 1344. 1359. 1187. 1554. 1459. 1303. 1390.]
 [2460. 1403. 1344.    0. 1406. 1288. 1480. 1471. 1391. 1415.]
 [2435. 1372. 1359. 1406.    0. 1279. 1588. 1544. 1298. 1352.]
 [2253. 1351. 1187. 1288. 1279.    0. 1531. 1352. 1296. 1358.]
 [2703. 1655. 1554. 1480. 1588. 1531.    0. 1624. 1638. 1570.]
 [2500. 1563. 1459. 1471. 1544. 1352. 1624.    0. 1558. 1430.]
 [2422. 1368. 1303. 1391. 1298. 1296. 1638. 1558.    0. 1384.]
 [2489. 1490. 1390. 1415. 1352. 1358. 1570. 1430. 1384.    0.]]


### KS

In [36]:
def ks(models, x_test):
    num_samples = x_test.shape[0]
    num_models = len(models)

    # مرحله 1: پیش‌بینی کلاس‌ها برای داده‌های تست توسط هر مدل
    argmax_preds = np.array([model.predict(x_test).argmax(axis=1) for model in client_models])

    # مرحله 2: یافتن کلاس پرتکرار برای هر نمونه
    most_frequent_classes = [Counter(argmax_preds[:, i]).most_common(1)[0][0] for i in range(num_samples)]

    # مرحله 3: استخراج احتمال کلاس پرتکرار از تمامی مدل‌ها برای هر نمونه
    all_probabilities = np.zeros((num_samples, num_clients))
    for i, model in enumerate(client_models):
        predictions = model.predict(x_test)  # احتمالات کلاس‌های مختلف برای هر نمونه
        for j, sample in enumerate(x_test):
            most_frequent_class = most_frequent_classes[j]
            all_probabilities[j, i] = predictions[j, most_frequent_class]  # احتمال کلاس پرتکرار
    # مرحله 4: انجام تست KS بین مدل‌های کلاینت و ایجاد ماتریس KS
    ks_distance_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            # تست KS برای مقایسه احتمالات دو مدل کلاینت
            ks_statistic, _ = ks_2samp(all_probabilities[:, i], all_probabilities[:, j])
            ks_distance_matrix[i, j] = ks_statistic
            ks_distance_matrix[j, i] = ks_statistic  # ماتریس متقارن

    return ks_distance_matrix

# اجرای تابع با تعیین تعداد ارقام اعشار
matrix_ks = ks(client_models, x_test)
print("Distance Matrix for KS:")
print(matrix_ks)

# start_time = time.time()
# end_time = time.time()
# execution_time = end_time - start_time
# print(f"\nExecution time for delta_class: {execution_time:.4f} seconds")

Distance Matrix for KS:
[[0.     0.7898 0.7769 0.7812 0.7726 0.7851 0.7921 0.7846 0.7841 0.7788]
 [0.7898 0.     0.0359 0.0487 0.0707 0.0532 0.0683 0.0324 0.0165 0.0207]
 [0.7769 0.0359 0.     0.0422 0.0995 0.0504 0.0584 0.0396 0.0364 0.027 ]
 [0.7812 0.0487 0.0422 0.     0.109  0.0756 0.0292 0.0226 0.0477 0.0503]
 [0.7726 0.0707 0.0995 0.109  0.     0.0651 0.128  0.0873 0.0656 0.0886]
 [0.7851 0.0532 0.0504 0.0756 0.0651 0.     0.094  0.0631 0.0431 0.0375]
 [0.7921 0.0683 0.0584 0.0292 0.128  0.094  0.     0.0457 0.0664 0.0671]
 [0.7846 0.0324 0.0396 0.0226 0.0873 0.0631 0.0457 0.     0.0261 0.0392]
 [0.7841 0.0165 0.0364 0.0477 0.0656 0.0431 0.0664 0.0261 0.     0.0246]
 [0.7788 0.0207 0.027  0.0503 0.0886 0.0375 0.0671 0.0392 0.0246 0.    ]]


### Chi2

In [37]:
def chi_square(models, x_test):
    num_models = len(models)
    num_samples = x_test.shape[0] # تعداد نمونه‌ها  
    n_classes = 10  # تعداد کلاس ها

    # ساخت ماتریس متقارن برای نتایج Chi-Square
    chi2_matrix = np.zeros((num_models, num_models))
    all_probabilities = np.zeros((num_samples, num_models))

    for i, model in enumerate(models):
        predictions = model.predict(x_test)  # احتمالات کلاس‌های مختلف برای هر نمونه
        for j, sample in enumerate(x_test):
            all_probabilities[j, i] = predictions[j].argmax()

    # مرحله: ایجاد جدول‌های متقاطع و انجام تست Chi-Square
    for i in range(num_models):
        for j in range(i + 1, num_models):
            # ساخت جدول متقاطع
            contingency_table = pd.crosstab(all_probabilities[:, i], all_probabilities[:, j])
            chi2_stat, p_value, _, _ = chi2_contingency(contingency_table)
            chi2_matrix[i, j] = chi2_stat
            chi2_matrix[j, i] = chi2_stat  # ماتریس متقارن

    return chi2_matrix

# اجرای تابع با تعیین تعداد ارقام اعشار
matrix_chi2 = chi_square(client_models, x_test)
print("Distance Matrix for Chi square:")
print(matrix_chi2)

# start_time = time.time()
# end_time = time.time()
# execution_time = end_time - start_time
# print(f"\nExecution time for delta_class: {execution_time:.4f} seconds")

Distance Matrix for Chi square:
[[    0.         59254.46147014 60981.72176082 60273.98429588
  59588.56213485 61156.46079432 58658.79557984 60344.67810457
  60539.66473789 60487.85914148]
 [59254.46147014     0.         79361.84315696 79162.47486044
  79536.89488198 79954.74428068 77608.67620493 77822.72137767
  79714.5580719  78569.49731061]
 [60981.72176082 79361.84315696     0.         80410.25267403
  79531.26101984 81280.48592136 78501.10363998 79962.93300736
  80764.94613892 79748.006255  ]
 [60273.98429588 79162.47486044 80410.25267403     0.
  78677.54376724 79474.33080307 78862.8591814  79039.91029528
  79431.76530798 79161.25960421]
 [59588.56213485 79536.89488198 79531.26101984 78677.54376724
      0.         79380.56117829 76975.00455101 77922.70022703
  79540.25785321 79334.63434682]
 [61156.46079432 79954.74428068 81280.48592136 79474.33080307
  79380.56117829     0.         77948.97989547 80250.30392468
  80004.87347588 79392.77737483]
 [58658.79557984 77608.67620493 78

### The problematic client

In [40]:
def meancal(matrix):
    temp = 0
    x = matrix.shape[0]    
    arrmean = []
    
    for i in range(0,x):
        #print(matrix[i].mean())
        temp = (matrix[i].mean())    
        arrmean.append(temp)
    return arrmean

def iqrfunc(nparray):
    data = np.array(nparray)
    q1 = np.percentile(data,25)
    q3 = np.percentile(data,75)
    iqr = q3 -q1
    lower_bound = q1-(iqr*1.5)
    upper_bound = q3+(iqr*1.5)
    outliers = np.where((data<lower_bound) | (data>upper_bound))[0]
    return outliers
    
diff_class = delta_class(client_models, x_test)
diff_score = delta_score(client_models, x_test,0.1)
diff_ks = ks(client_models, x_test)
diff_chi2 = chi_square(client_models, x_test)

temp_c = meancal(diff_class)
temp_s = meancal(diff_score)
temp_ks = meancal(diff_ks)
temp_chi2 = meancal(diff_chi2)

print(f"Problemmatic Clients for Delta class {iqrfunc(temp_c)}:")                       
print(f"Problemmatic Clients for Delta Score {iqrfunc(temp_s)}:")                                            
print(f"Problemmatic Clients for KS {iqrfunc(temp_ks)}:")                       
print(f"Problemmatic Clients for X2 {iqrfunc(temp_chi2)}:") 

Problemmatic Clients for Delta class [0 6]:
Problemmatic Clients for Delta Score [0]:
Problemmatic Clients for KS [0]:
Problemmatic Clients for X2 [0 6]:


### Plots

In [ ]:
# import numpy as np

# # فرض کنید این لیست کلاینت‌های باگ‌دار واقعی است
# # ابتدا از خروجی تحلیل‌ها برای شناسایی کلاینت‌های باگ‌دار استفاده می‌کنیم
# true_problematic_clients = []

# # برای هر نوع تحلیل
# for title, diff in zip(["Delta class", "Delta Score", "KS", "X2"], 
#                        [temp_c, temp_s, temp_ks, temp_chi2]):
    
#     # تبدیل لیست به آرایه NumPy
#     diff_array = np.array(diff)  
    
#     # ایندکس کلاینت‌های باگ‌دار
#     problematic_clients = iqrfunc(meancal(diff_array))
#     print(f"Problemmatic Clients for {title} {problematic_clients}:")
    
#     # به‌روزرسانی لیست کلاینت‌های باگ‌دار واقعی
#     true_problematic_clients.extend(problematic_clients.tolist())

# # تبدیل به آرایه numpy و حذف تکراری‌ها
# true_problematic_clients = np.unique(np.array(true_problematic_clients))

# # توابع برای محاسبه معیارها
# def precision(predicted, actual):
#     tp = len(np.intersect1d(predicted, actual))  # True Positives
#     fp = len(np.setdiff1d(predicted, actual))     # False Positives
#     return tp / (tp + fp) if (tp + fp) > 0 else 0

# def recall(predicted, actual):
#     tp = len(np.intersect1d(predicted, actual))  # True Positives
#     fn = len(np.setdiff1d(actual, predicted))     # False Negatives
#     return tp / (tp + fn) if (tp + fn) > 0 else 0

# def f1_score(predicted, actual):
#     p = precision(predicted, actual)
#     r = recall(predicted, actual)
#     return 2 * (p * r) / (p + r) if (p + r) > 0 else 0

# def accuracy(predicted, actual, total_clients):
#     tp = len(np.intersect1d(predicted, actual))  # True Positives
#     tn = total_clients - len(actual) - len(predicted) + len(np.intersect1d(predicted, actual))  # True Negatives
#     return (tp + tn) / total_clients

# # محاسبه معیارها برای هر خروجی
# for title, diff in zip(["Delta class", "Delta Score", "KS", "X2"], 
#                        [temp_c, temp_s, temp_ks, temp_chi2]):
    
#     # تبدیل لیست به آرایه NumPy
#     diff_array = np.array(diff)
    
#     # ایندکس کلاینت‌های باگ‌دار
#     problematic_clients = iqrfunc(meancal(diff_array))
#     print(f"\nProblemmatic Clients for {title} {problematic_clients}:")
    
#     # محاسبه معیارها
#     p = precision(problematic_clients, true_problematic_clients)
#     r = recall(problematic_clients, true_problematic_clients)
#     f1 = f1_score(problematic_clients, true_problematic_clients)
#     acc = accuracy(problematic_clients, true_problematic_clients, len(client_models))

#     # نمایش نتایج
#     print(f"Precision: {p:.2f}, Recall: {r:.2f}, F1 Score: {f1:.2f}, Accuracy: {acc:.2f}\n")


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt


# # تابع محاسبه میانگین
# def meancal(matrix):
#     x = matrix.shape[0]    
#     arrmean = []
#     for i in range(x):
#         temp = ((matrix[i].mean()) * 10) / 9
#         arrmean.append(temp)
#     return arrmean

# # محاسبه میانگین‌ها برای ماتریس‌های مختلف
# temp_c = meancal(diff_class)
# temp_s = meancal(diff_score)
# temp_ks = meancal(diff_ks)
# temp_chi2 = meancal(diff_chi2)

# # داده‌ها و برچسب‌ها برای رسم نمودار
# data = [temp_c, temp_s, temp_ks, temp_chi2]
# labels = ['diff_class', 'diff_score', 'diff_ks', 'diff_chi2']

# # رسم نمودار box plot
# plt.boxplot(data, labels=labels)
# plt.title('Boxplot of Calculated Means')
# plt.ylabel('Values')
# plt.show()


In [13]:
# import numpy as np
# import matplotlib.pyplot as plt

# # فرضیات: جایگزین‌های داده‌های diff_class, diff_score, diff_ks, diff_chi2
# np.random.seed(42)  # برای بازتولید پذیری
# diff_class = np.random.rand(5, 10)  # ماتریس تصادفی 5x10 برای نمایش
# diff_score = np.random.rand(5, 10)  # ماتریس تصادفی 5x10
# diff_ks = np.random.rand(5, 10)  # ماتریس تصادفی 5x10
# diff_chi2 = np.random.rand(5, 10)  # ماتریس تصادفی 5x10

# # تابع محاسبه میانگین
# def meancal(matrix):
#     x = matrix.shape[0]    
#     arrmean = []
#     for i in range(x):
#         temp = ((matrix[i].mean()) * 10) / 9
#         arrmean.append(temp)
#     return arrmean

# # محاسبه میانگین‌ها برای ماتریس‌های مختلف
# temp_c = meancal(diff_class)
# temp_s = meancal(diff_score)
# temp_ks = meancal(diff_ks)
# temp_chi2 = meancal(diff_chi2)

# # نمودار برای diff_class
# plt.figure()
# plt.boxplot(temp_c)
# plt.title('Boxplot of diff_class')
# plt.ylabel('Values')
# plt.show()

# # نمودار برای diff_score
# plt.figure()
# plt.boxplot(temp_s)
# plt.title('Boxplot of diff_score')
# plt.ylabel('Values')
# plt.show()

# # نمودار برای diff_ks
# plt.figure()
# plt.boxplot(temp_ks)
# plt.title('Boxplot of diff_ks')
# plt.ylabel('Values')
# plt.show()

# # نمودار برای diff_chi2
# plt.figure()
# plt.boxplot(temp_chi2)
# plt.title('Boxplot of diff_chi2')
# plt.ylabel('Values')
# plt.show()
